# preprocessing
- find null values
- replace with feature mean
- find outliers (especially m2)
- enumarate categorical features
- drop title col
- drop id col
- convert all prices to try
- drop lat lon
- convert date values to be of the same race
- drop type (bcz all values are flat)
- drop currency
- remove outlier prices (25000 TL, 8500000TL)

In [ ]:

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
FILENAME = "real_estate_data.csv"
df = pd.read_csv(FILENAME)
df

In [ ]:
# Find rows where 'bath' column has null values
null_bath_rows = df[df['bath'].isnull()]
null_bath_rows.isnull().count()
df[df["rooms"] == "Unknown"]

# Set rooms property

In [ ]:
# Convert 'm2' column to numeric if it's not already
df['m2'] = pd.to_numeric(df['m2'], errors='coerce')

for i in df[df["rooms"] == "Unknown"].index:
    m2 = df.iloc[i]["m2"]
    mode = df[df['m2'] == m2]["rooms"].mode()
    if not mode.empty:  # Check if mode is not empty
        mode_value = mode.iloc[0]  # Take the first mode value
        df.at[i, "rooms"] = mode_value

# Set residence property

In [ ]:
df['resid'] = (df['resid'] == 'unknown') & (df['price'] > 150000)

In [ ]:
# Replace 'None' with np.nan if it's not already done
new_df = df.replace('None', np.NaN)

# Remove rows with any null values
df_cleaned = new_df.dropna()

# Save the cleaned DataFrame to a new CSV file
df_cleaned.to_csv('cleaned_data.csv', index=False)

# Display the cleaned DataFrame

df_cleaned



In [ ]:

df = pd.read_csv('cleaned_data.csv')



In [ ]:
missing_values_summary = df.isnull().sum()
missing_values_summary


# DROP FETURES

## Drop lat - lon - type - title - id - due and Status features from dataframe

In [ ]:
if 'lat' in df.columns:
    df.drop('lat', axis=1, inplace=True)

if 'lon' in df.columns:
    df.drop('lon', axis=1, inplace=True)

if 'type' in df.columns:
    df.drop('type', axis=1, inplace=True)

if 'title' in df.columns:
    df.drop('title', axis=1, inplace=True)

if 'Id' in df.columns:
    df.drop('Id', axis=1, inplace=True)

if 'due' in df.columns:
    df.drop('due', axis=1, inplace=True)

if 'Status' in df.columns:
    df.drop('Status', axis=1, inplace=True)

df

## Change all currency with turkish lira equivalent and drop currency feature

In [ ]:

if "currency" in df.columns:

    df.loc[df['currency'] == 'Euro', 'price'] *= 4
    df.loc[df['currency'] == 'US Dollar', 'price'] *= 3.5
    df.loc[df['currency'] == 'British Pound', 'price'] *= 4.5

    df.drop("currency", axis=1, inplace=True)
df["price"]




## Outlier detection using confidence interval

In [ ]:
Q1 = df['price'].quantile(0.25)
Q3 = df['price'].quantile(0.75)
IQR = Q3 - Q1

# Define a condition to identify outliers
outlier_condition = (df['price'] < (Q1 - 1.5 * IQR)) | (df['price'] > (Q3 + 1.5 * IQR))

# sadece turk lirasinda karsilasilan bir durum
# Display rows containing outliers
outliers = df[outlier_condition]
df.drop(outliers.index, inplace=True)
df


## Date transformation

In [ ]:
# df.loc()

# for tr, en in turkish_months.values():
#     df.loc[df['date'].str.contains(tr), 'price']

aylar = {"Ocak": "01", "Şubat": "02", "Mart": "03", "Nisan": "04", "Mayıs": "05", "Haziran": "06",
             "Temmuz": "07", "Ağustos": "08", "Eylül": "09", "Ekim": "10", "Kasım": "11", "Aralık": "12"}

def transform_date(date):
    match date:
        case str():
            result = '-'.join(date.split()[::-1])
            for ay, ay_kodu in aylar.items():
                result = result.replace(ay, ay_kodu)
            
            return result
        case _:
            return date

def transform_date_to_ms(date_str) -> int:
    from dateutil import parser

    match date_str:
        case str():
            return parser.parse(date_str, dayfirst=True).timestamp().__floor__()
        case _:
            return int(date_str)


# 'date' sütununu dönüştürün
df['date'] = df['date'].apply(transform_date)
df['date'] = df['date'].apply(transform_date_to_ms)
df['date']

## Bath transformation

In [ ]:
def transform_bath(bath):
    match bath:
        case str():
            if "+" in bath:
                return float(bath.replace("+", ""))
            else:
                return float(bath)
        case _:
            return bath

df["bath"] = df["bath"].apply(transform_bath)
df

In [ ]:
df["rooms"].value_counts()

# Enumeration Process

In [ ]:
from pandas import DataFrame
from sklearn.preprocessing import LabelEncoder

def enumerate_column(column: str, df: DataFrame):
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])

columns_to_enumerate = ["loc city", "loc county", "loc dist", "rooms", "age", "floor", "heat", ""]
df.dtypes


## Date converted from string to epoch ms

In [ ]:
df['price'] = pd.to_numeric(df['price'], errors='coerce')

# Create a box plot for the 'price' column
sns.set(style="whitegrid")
plt.figure(figsize=(8, 6))
sns.boxplot(x='price', data=df, orient='v')
plt.title('Box Plot for Price')
plt.show()

In [ ]:
obj = (df.dtypes == 'object')
object_cols = list(obj[obj].index)
print("Categorical variables:",len(object_cols))

int_ = (df.dtypes == 'int')
num_cols = list(int_[int_].index)
print("Integer variables:",len(num_cols))

fl = (df.dtypes == 'float')
fl_cols = list(fl[fl].index)
print("Float variables:",len(fl_cols))

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Make copy to avoid changing original data

# Apply label encoder to each column with categorical data
label_encoder = LabelEncoder()
for col in object_cols:
    df[col] = label_encoder.fit_transform(df[col].astype(str))


df

In [ ]:
plt.figure(figsize=(12, 6))
sns.heatmap(df.corr(),
			cmap = 'BrBG',
			fmt = '.2f',
			linewidths = 2,
			annot = True)

In [ ]:
# plt.figure(figsize=(18, 36))
# plt.title('Categorical Features: Distribution')
# plt.xticks(rotation=90)
# index = 1

# for col in object_cols:
# 	y = df[col].value_counts()
# 	plt.subplot(11, 4, index)
# 	plt.xticks(rotation=90)
# 	sns.barplot(x=list(y.index), y=y)
# 	index += 1

In [ ]:
# Split

In [ ]:
from sklearn.model_selection import train_test_split

# Separate target from predictors
y = df.price
X = df.drop('price', axis=1)

# Divide data into training and validation subsets
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [ ]:


from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

# Get names of columns with missing values
cols_with_missing = [col for col in X_train.columns
                     if X_train[col].isnull().any()]

# Drop columns in training and validation data
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_test.drop(cols_with_missing, axis=1)

print("MAE (Drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_test))


In [ ]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
# Import necessary libraries
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Create and train the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate and print the mean squared error
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

train_accuracy = model.score(X_train, y_train)  # R-squared or another appropriate metric
print(f'Train Accuracy: {train_accuracy}')

test_accuracy = model.score(X_test, y_test)  # R-squared or another appropriate metric
print(f'Test Accuracy: {test_accuracy}')


this can only mean one thing: *Linear regression is unefficient for this dataset*

In [ ]:
# decision tree regression

from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

# Create and train the decision tree regression model
tree = DecisionTreeRegressor()
tree.fit(X_train, y_train)

# Make predictions on the test set
y_pred = tree.predict(X_test)

# Calculate and print the mean squared error
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

train_accuracy = tree.score(X_train, y_train)  # R-squared or another appropriate metric
print(f'Train Accuracy: {train_accuracy}')

test_accuracy = tree.score(X_test, y_test)  # R-squared or another appropriate metric
print(f'Test Accuracy: {test_accuracy}')